In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime

In [2]:
from datetime import datetime

import pandas as pd
from prettytable import PrettyTable


class DataManager:
    @classmethod
    def show_status(self, df: pd.DataFrame):
        table = PrettyTable(
            ["Variable", "Type", "Missing Values", "Duplicates", "Outliers"]
        )

        for col in df.columns:
            # データ型の判定
            if pd.api.types.is_numeric_dtype(df[col]):
                col_type = "Numerical"
            else:
                col_type = "Categorical"

            # 欠損値
            missing_values = df[col].isnull().sum()

            # 重複数
            duplicates = df.duplicated(subset=[col]).sum()

            # 外れ値（数値型のみ）
            if col_type == "Numerical":
                mean = df[col].mean()
                std = df[col].std()
                outliers = ((df[col] - mean).abs() > 3 * std).sum()
            else:
                outliers = "N/A"

            table.add_row([col, col_type, missing_values, duplicates, outliers])
        print(table)

    @classmethod
    def extract_term_data(
        cls, target_col: str, df: pd.DataFrame, start_time: datetime, end_time: datetime
    ):
        df[target_col] = pd.to_datetime(df[target_col])
        filtered_df = df[(df[target_col] >= start_time) & (df[target_col] <= end_time)]
        return filtered_df

    @classmethod
    def check_date_sequence(cls, df: pd.DataFrame, date_col: str = "date"):
        dates = pd.to_datetime(df[date_col]).sort_values().reset_index(drop=True)
        expected = pd.date_range(start=dates.min(), end=dates.max(), freq="D")
        missing = expected.difference(dates)
        return {
            "is_continuous": len(missing) == 0,
            "missing_dates": missing.strftime("%Y-%m-%d").tolist(),
        }

In [3]:
base_df = pd.read_csv("data/base/hourly_filled.csv")
base_df["Datetime_hour"] = pd.to_datetime(base_df["Datetime_hour"])
base_df = base_df.set_index("Datetime_hour", drop=True)


def split_df(df, term):
    _df = df.copy()
    historical_df = _df[_df.index < term]
    validate_df = _df[_df.index >= term]
    return historical_df, validate_df


# ===== 予測したい時刻 =====
t0 = pd.Timestamp("2025-09-10 07:00:00")
tend = pd.Timestamp("2025-09-11 07:00:00")
# ===== 学習/検証に分割（t0を境に“過去”と“将来”）=====
historical_df, validate_df = split_df(base_df, datetime(2025, 9, 10, 7))

In [4]:
from deep_reinforcement_learning.environment.control_env import AirControlEnv
from deep_reinforcement_learning.environment.prediction.model import load_residual_model

In [5]:
model = load_residual_model("models/xgb_weight.joblib")
Env = AirControlEnv(
    model=model,
    base_df=historical_df,
    start_term=t0,
    end_term=tend,
    weather_forecast=validate_df,
)

In [6]:
print(Env.start_term)
obs, info = Env.reset()

2025-09-10 07:00:00


In [7]:
print(len(obs[0]))
obs

133


array([[18.      , 26.      , 21.      , 23.5     , 23.      , 23.75    ,
        24.      , 24.      , 24.      , 24.      , 22.      , 22.      ,
        22.      , 24.      , 24.      , 23.      , 23.      , 27.      ,
        24.      , 26.      , 26.      , 26.      , 24.      , 26.      ,
        21.      , 26.      ,  1.      ,  1.      ,  0.      ,  0.      ,
         1.      ,  1.      ,  1.      ,  1.      ,  1.      ,  1.      ,
         1.      ,  1.      ,  0.      ,  1.      ,  1.      ,  1.      ,
         1.      ,  1.      ,  1.      ,  1.      ,  0.      ,  0.      ,
         1.      ,  1.      ,  0.      ,  0.      ,  2.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  1.      ,  1.      ,  1.      ,
         1.      ,  1.      ,  2.      ,  2.      ,  0.      ,  2.      ,
         0.      ,  2.      ,  2.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  1.      ,  1.      ,  0.      ,  0.      ,
         1.      ,  1.      ,  0.     

In [8]:
import torch

act = torch.Tensor(
    [
        [
            1,
            0,
            0,
            0,
            2,
            2,
            3,
            1,
            1,
            1,
            2,
            1,
            2,
            0,
            0,
            0,
            1,
            0,
            0,
            0,
            3,
            0,
            0,
            0,
            1,
            0,
            0,
            0,
            1,
            0,
            2,
            1,
            3,
            2,
            2,
            1,
            2,
            0,
            0,
            0,
            1,
            0,
            0,
            0,
            2,
            1,
            1,
            1,
            1,
            1,
            2,
            1,
            3,
            0,
            0,
            1,
            2,
            1,
            3,
            1,
            2,
            0,
            1,
            1,
            1,
            0,
            0,
            0,
            3,
            1,
            0,
            1,
            3,
            1,
            1,
            1,
            2,
            0,
            0,
            0,
            3,
            1,
            3,
            1,
            3,
            0,
            0,
            0,
            3,
            0,
            2,
            1,
            1,
            0,
            0,
            0,
            1,
            1,
            3,
            1,
            1,
            0,
            0,
            0,
        ]
    ]
)

obs, reward, terminated, truncated, info = Env.step(act)

In [9]:
print(len(obs[0]))
obs

133


array([[19.5     , 20.      , 19.5     , 20.      , 19.5     , 20.5     ,
        19.5     , 19.5     , 20.5     , 20.      , 19.5     , 20.      ,
        19.5     , 20.5     , 20.      , 20.      , 19.5     , 20.5     ,
        20.5     , 20.      , 20.5     , 20.5     , 20.5     , 19.5     ,
        19.5     , 19.5     ,  0.      ,  2.      ,  1.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  2.      ,  0.      ,
         0.      ,  1.      ,  1.      ,  0.      ,  1.      ,  0.      ,
         0.      ,  1.      ,  1.      ,  0.      ,  1.      ,  0.      ,
         0.      ,  0.      ,  1.      ,  0.      ,  0.      ,  3.      ,
         2.      ,  0.      ,  0.      ,  0.      ,  0.      ,  2.      ,
         2.      ,  0.      ,  0.      ,  1.      ,  2.      ,  0.      ,
         3.      ,  1.      ,  0.      ,  0.      ,  1.      ,  0.      ,
         3.      ,  0.      ,  2.      ,  0.      ,  3.      ,  0.      ,
         0.      ,  1.      ,  1.     

In [10]:
info

{'current_temp_index': array([[1, 2, 1, 2, 1, 3, 1, 1, 3, 2, 1, 2, 1, 3, 2, 2, 1, 3, 3, 2, 3, 3,
         3, 1, 1, 1]]),
 'time': Timestamp('2025-09-10 08:00:00'),
 'reward': -4.95378755677509,
 'kwh': 320438.75467470713,
 'discomfort': 8.737589526200011,
 'kwh_hat': 3.511368979806669,
 'disc_hat': 2.5938419906511925,
 'kwh_ref': 91257.5,
 'disc_ref': 3.368589743589744,
 'cost_term': 0.9982188245431174,
 'comfort_term': 3.9555687322319733}